<a href="https://colab.research.google.com/github/KonstantinKazakov11/DA.Homeworks/blob/main/ML1_titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import pandas as pd
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from google.colab import files
uploaded = files.upload()

Saving titanicdf.csv to titanicdf.csv


In [18]:
RANDOM_SEED = 777

In [49]:
titanic = pd.read_csv('/titanicdf.csv')

In [50]:
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [51]:
titanic.describe()


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [52]:
titanic.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [53]:
titanic.loc[titanic.Age.isna(),'Age'] = titanic['Age'].mean()
titanic.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [54]:
titanic = titanic[['Survived', 'Pclass', 'Sex', 'Age', 'Fare']]

In [55]:
#Закодируем пол в число
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [56]:
titanic['POL'] = le.fit_transform(titanic['Sex'])
titanic

,Survived,Pclass,Sex,Age,Fare,POL
0,0,3,male,22.000000,7.2500,1
1,1,1,female,38.000000,71.2833,0
2,1,3,female,26.000000,7.9250,0
3,1,1,female,35.000000,53.1000,0
4,0,3,male,35.000000,8.0500,1
...,...,...,...,...,...,...
886,0,2,male,27.000000,13.0000,1
887,1,1,female,19.000000,30.0000,0
888,0,3,female,29.699118,23.4500,0
889,1,1,male,26.000000,30.0000,1


In [57]:
titanic_orange = titanic[['Survived', 'Pclass', 'POL', 'Age', 'Fare']]
titanic_orange.to_csv('titanic_orange.csv')
X_train, X_test, y_train, y_test= train_test_split(titanic.drop(['Age', 'Sex'], axis=1), 
                                                  titanic['Age'],
                                                  test_size = 0.3,
                                                  random_state = RANDOM_SEED)
print(X_train.shape,X_test.shape)

(623, 4) (268, 4)


In [58]:
#С помощью линейной регрессии
clf = LinearRegression()
clf.fit(X_train,y_train)

LinearRegression()

In [59]:
pred = clf.predict(X_test)

In [60]:
#С помощью деревьев
tree_regressor = DecisionTreeRegressor()
tree_regressor.fit(X_train,y_train)

DecisionTreeRegressor()

In [61]:
predict_tree = tree_regressor.predict(X_test)
tree_regressor.feature_importances_

array([0.08811586, 0.17620338, 0.63252515, 0.10315561])

In [62]:
frame = {'Важность параметра':tree_regressor.feature_importances_, 'Фича':X_train.columns}
result = pd.DataFrame(frame)
result.sort_values(by='Важность параметра', ascending=False)

,Важность параметра,Фича
2,0.632525,Fare
1,0.176203,Pclass
3,0.103156,POL
0,0.088116,Survived


In [63]:
#Сравниваем результаты
df_with_predict = X_test
df_with_predict['real_value'] = y_test
df_with_predict['predict_LR'] = pred
df_with_predict['predict_DT'] = predict_tree
df_with_predict[['real_value','predict_LR', 'predict_DT']]

,real_value,predict_LR,predict_DT
119,2.000000,27.142655,18.000000
429,32.000000,22.109711,24.500000
598,29.699118,27.598365,31.979647
275,63.000000,34.058425,36.000000
120,21.000000,31.837949,23.666667
...,...,...,...
469,0.750000,22.235451,14.500000
697,29.699118,22.757284,29.699118
534,30.000000,28.166511,20.000000
752,33.000000,27.495357,26.000000


In [64]:
print('MAE on Linear Regression', mean_absolute_error(y_test, pred))
print('MAE on DecisionTreeRegressor', mean_absolute_error(y_test, predict_tree))

MAE on Linear Regression 9.426669947988948
MAE on DecisionTreeRegressor 10.399653106142955
